In [ ]:
import pyarrow.dataset as ds

In [ ]:


# split = "nuplan_private_test"
# dataset = ds.dataset(f"/home/daniel/asim_workspace/data/{split}", format="ipc")

# fragments = list(dataset.get_fragments())
# print(f"Number of files: {len(fragments)}")

In [ ]:
# filtered_data

In [ ]:
s3_uri = "asimtesting/nuplan_private_test/2021.07.25.16.16.23_veh-26_02446_02589.arrow"

In [ ]:
import pyarrow as pa
import pyarrow.fs as fs

import os


s3_fs = fs.S3FileSystem(
    access_key=os.environ.get('AWS_ACCESS_KEY_ID'),
    secret_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
    region=os.environ.get('AWS_DEFAULT_REGION')
)
# with s3_fs.open_input_file(s3_uri) as file:
#     table = pa.ipc.open_file(file).read_all()
#     print(table)


In [ ]:
import pyarrow.dataset as ds
import pyarrow as pa



# Create dataset pointing to the specific file
dataset = ds.dataset(f"{s3_uri}", format="ipc", filesystem=s3_fs)

# Get all column names and remove the ones you want to drop
all_columns = dataset.schema.names
columns_to_keep = [col for col in all_columns if col not in ["front_cam_demo", "front_cam_transform"]]

# Load only the columns you want (more memory efficient)
table = dataset.to_table(columns=columns_to_keep)

# Save locally
with pa.ipc.new_file("filtered_file.arrow", table.schema) as writer:
    writer.write_table(table)

In [ ]:
# Load only the first 10 rows and only the columns you want
  # Display first 10 rows
# table = dataset.take([i for i in range(0, 100, 1)], columns=columns_to_keep)
table = dataset.take([i for i in range(0, 100, 1)], columns=all_columns)

# Save locally
with pa.ipc.new_file("filtered_file_v2.arrow", table.schema) as writer:
    writer.write_table(table)


In [ ]:
with pa.memory_map("filtered_file_v2.arrow", "r") as source:
    mmap_table = pa.ipc.open_file(source).read_all()

In [ ]:
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.dataset as ds
import pyarrow as pa
from huggingface_hub import hf_hub_download
from io import BytesIO

# Download specific file from the repo
file_path = hf_hub_download(
    repo_id="DanielDauner/delete_me",
    filename="2021.05.25.14.16.10_veh-35_01690_02183.arrow",  # or whatever the file is named
    repo_type="dataset"
)

# Read with PyArrow
with pa.memory_map(file_path, "r") as source:
    mmap_table = pa.ipc.open_file(source).read_all()

# Create dataset and slice
sliced_table = mmap_table.slice(0, 1000)  # First 1000 rows
sliced_table